<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

In [ ]:
import numpy as np
from scipy.optimize import quadratic_assignment
A = np.array([[0, 80, 150, 170], [80, 0, 130, 100],
              [150, 130, 0, 120], [170, 100, 120, 0]])
B = np.array([[0, 5, 2, 7], [0, 0, 3, 8],
              [0, 0, 0, 3], [0, 0, 0, 0]])
res = quadratic_assignment(A, B)
print(res)

     fun: 3260
 col_ind: [0 3 2 1]
     nit: 9

The see the relationship between the returned ``col_ind`` and ``fun``,
use ``col_ind`` to form the best permutation matrix found, then evaluate
the objective function $f(P) = trace(A^T P B P^T )$.


In [ ]:
perm = res['col_ind']
P = np.eye(len(A), dtype=int)[perm]
fun = np.trace(A.T @ P @ B @ P.T)
print(fun)

3260

Alternatively, to avoid constructing the permutation matrix explicitly,
directly permute the rows and columns of the distance matrix.


In [ ]:
fun = np.trace(A.T @ B[perm][:, perm])
print(fun)

3260

Although not guaranteed in general, ``quadratic_assignment`` happens to
have found the globally optimal solution.


In [ ]:
from itertools import permutations
perm_opt, fun_opt = None, np.inf
for perm in permutations([0, 1, 2, 3]):
    perm = np.array(perm)
    fun = np.trace(A.T @ B[perm][:, perm])
    if fun < fun_opt:
        fun_opt, perm_opt = fun, perm
print(np.array_equal(perm_opt, res['col_ind']))

True

Here is an example for which the default method,
:ref:`'faq' <optimize.qap-faq>`, does not find the global optimum.


In [ ]:
A = np.array([[0, 5, 8, 6], [5, 0, 5, 1],
              [8, 5, 0, 2], [6, 1, 2, 0]])
B = np.array([[0, 1, 8, 4], [1, 0, 5, 2],
              [8, 5, 0, 5], [4, 2, 5, 0]])
res = quadratic_assignment(A, B)
print(res)

     fun: 178
 col_ind: [1 0 3 2]
     nit: 13

If accuracy is important, consider using  :ref:`'2opt' <optimize.qap-2opt>`
to refine the solution.


In [ ]:
guess = np.array([np.arange(len(A)), res.col_ind]).T
res = quadratic_assignment(A, B, method="2opt",
                           options = {'partial_guess': guess})
print(res)

     fun: 176
 col_ind: [1 2 3 0]
     nit: 17